In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time # Adicionando time para o sleep e teste de limite

In [4]:
# URL principal e cabeçalhos
URL_BASE = "https://cpisp.org.br/direitosquilombolas/observatorio-terras-quilombolas/"
CABECALHO = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/555.36 (KHTML, like Gecko) Chrome/99.0.4472.124 Safari/555.36'
}

# -----------------------------------------------------------
# Funções de Coleta (obter_links_comunidades
# -----------------------------------------------------------
def obter_links_comunidades(url_base):
    print("Acessando a página principal para buscar links...")
    links_comunidades = []
    
    try:
        response = requests.get(url_base, headers=CABECALHO, timeout=15)
        response.raise_for_status() 
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL base: {e}")
        return links_comunidades

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Seletor baseado na imagem: <h2> com classe 'entry-title' contendo o <a>
    links_encontrados = soup.find_all('h2', class_='entry-title')
    
    for h2 in links_encontrados:
        link_tag = h2.find('a', href=True)
        if link_tag:
             # Usa urljoin para garantir que o URL seja absoluto
             full_url = urljoin(url_base, link_tag['href'])
             if full_url not in links_comunidades:
                links_comunidades.append(full_url)
    
    print(f"Encontrados {len(links_comunidades)} links de comunidades para rastrear.")
    return links_comunidades

# -----------------------------------------------------------
# Função de Extração (AGORA USANDO O SELETOR DA TABELA)
# -----------------------------------------------------------
def extrair_dados_comunidade(url_comunidade):
    """
    Acessa a página detalhada e extrai dados da tabela estruturada (<tr> e <td>).
    """
    dados = {
        'Comunidade(s)': '',
        'Município': '',
        'Unidade da federação': '',
        'População': '',
        'Situação fundiária': '',
        'Data da última atualização': '',
    }
    
    try:
        response = requests.get(url_comunidade, headers=CABECALHO, timeout=15)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url_comunidade}: {e}")
        return dados
    
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra a tabela que contém as informações gerais
    tabela_container = soup.find('div', class_='fusion-table table-1')
    
    if not tabela_container:
        # Tenta encontrar o nome da comunidade pelo título se a tabela falhar
        titulo_tag = soup.find('h1', class_='entry-title')
        if titulo_tag:
            dados['Comunidade(s)'] = titulo_tag.text.strip()
        return dados
    
    # Itera sobre as linhas da tabela (<tr>)
    for row in tabela_container.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) == 2:
            # A chave (rótulo) está na primeira célula, o valor na segunda
            key_raw = cells[0].text.strip().replace(':', '') # Remove dois pontos se houver
            value = cells[1].text.strip()
            
            # Mapeia o rótulo da página para o nome da coluna do DataFrame
            if key_raw == 'Comunidade(s)':
                dados['Comunidade(s)'] = value
            elif key_raw == 'Município':
                dados['Município'] = value
            elif key_raw == 'Unidade da federação':
                dados['Unidade da federação'] = value
            elif key_raw == 'População':
                dados['População'] = value
            elif key_raw == 'Situação fundiária':
                dados['Situação fundiária'] = value
            elif key_raw == 'Data da última atualização':
                dados['Data da última atualização'] = value

    # Tenta preencher o nome da comunidade a partir do título da página (H1) se o dado da tabela for vazio
    if not dados['Comunidade(s)']:
        titulo_tag = soup.find('h1', class_='entry-title')
        if titulo_tag:
            dados['Comunidade(s)'] = titulo_tag.text.strip()

    return dados

# -----------------------------------------------------------
# Função Principal
# -----------------------------------------------------------
def scraping_principal():
    links = obter_links_comunidades(URL_BASE)
    dados_completos = []
    
    if not links:
        print("Falha ao encontrar links. O seletor de links pode precisar de mais ajustes.")
        return pd.DataFrame()

    # --- FILTRO PARA TESTE (APENAS AS 5 PRIMEIRAS COMUNIDADES) ---
    #print("\n⚠️ Atenção: Limitando a coleta a 5 links para teste.") # descomentar para limitar a coleta a 5 links
    links_para_teste = links[:5]
    # -----------------------------------------------------------

    # Itera sobre os links e coleta os dados
    for i, link in enumerate(links): # Usar a variável links_para_teste no lugar de links, para limitar a coleta a 5 links
        print(f"[{i+1}/{len(links)}] Coletando dados de: {link}") # Usar a variável links_para_teste no lugar de links, para limitar a coleta a 5 links
        dados_completos.append(extrair_dados_comunidade(link))
        time.sleep(1) # Adiciona um pequeno atraso para ser gentil com o servidor
    
    df = pd.DataFrame(dados_completos)
    
    # Define a ordem das colunas e preenche vazios para manter a estrutura
    colunas_ordenadas = [
        'Comunidade(s)', 'Município', 'Unidade da federação', 'População', 
        'Situação fundiária', 'Data da última atualização'
    ]
    df = df.reindex(columns=colunas_ordenadas, fill_value='')

    return df

# Execução do script
tabela_quilombola = scraping_principal()

if not tabela_quilombola.empty:
    print("\n--- Tabela Final de Dados Coletados (Amostra de 5) ---")
    print(tabela_quilombola.to_markdown(index=False)) 
    print(f"\nTotal de {len(tabela_quilombola)} registros coletados no teste.")
else:
    print("\nO scraping não retornou dados. Verifique os seletores HTML ou o padrão Regex.")

Acessando a página principal para buscar links...
Encontrados 2115 links de comunidades para rastrear.
[1/2115] Coletando dados de: https://cpisp.org.br/2o-distrito-de-porto-grande-2/
[2/2115] Coletando dados de: https://cpisp.org.br/abacatal-aura/
[3/2115] Coletando dados de: https://cpisp.org.br/abelha-2/
[4/2115] Coletando dados de: https://cpisp.org.br/abobral-margem-direita/
[5/2115] Coletando dados de: https://cpisp.org.br/abobral-margem-esquerda/
[6/2115] Coletando dados de: https://cpisp.org.br/abobreiras/
[7/2115] Coletando dados de: https://cpisp.org.br/abolicao/
[8/2115] Coletando dados de: https://cpisp.org.br/acaua-2-2/
[9/2115] Coletando dados de: https://cpisp.org.br/achui/
[10/2115] Coletando dados de: https://cpisp.org.br/acorebela/
[11/2115] Coletando dados de: https://cpisp.org.br/acre/
[12/2115] Coletando dados de: https://cpisp.org.br/acre-cururupu/
[13/2115] Coletando dados de: https://cpisp.org.br/acude-jabuticatubas-mg/
[14/2115] Coletando dados de: https://cpis

In [5]:
# Supondo que 'tabela_quilombola' é o seu DataFrame final
tabela_quilombola.to_csv('dados_quilombolas_cpisp.csv', index=False, encoding='utf-8')

### carregando arquivo csv para análise estatística.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
tabela_quilombola = pd.read_csv('dados_quilombolas_cpisp.csv', encoding='utf-8')
tabela_quilombola.head()

,Comunidade(s),Município,Unidade da federação,População,Situação fundiária,Data da última atualização
0,"Itabatinga, Mangabeira, Porto Grande, Santo An...",Mocajuba,Pará,"1703 pessoas (Censo, 2022); 400 famílias",Titulada,07.09.2023
1,Abacatal – Aurá,Ananindeua,Pará,"368 pessoas (Censo, 2022) ; 53 famílias",Titulada,01.09.2023
2,Abelha,Carnaíba,Pernambuco,Sem informação,Não titulada,03.05.2023
3,Abobral Margem Direita,Eldorado,São Paulo,Sem informação,Não titulada,15.06.2023
4,Abobral,Eldorado,São Paulo,38 famílias (Itesp),Não titulada,05.03.2023


In [3]:
# 1. Pré-limpeza: Remove caracteres não visíveis (como o espaço não separável \xa0)
tabela_quilombola['Pop_Clean'] = tabela_quilombola['População'].str.replace(r'[^\w\s]', '', regex=True)

# 2. Extração: Captura a primeira sequência de UM OU MAIS dígitos (o número inteiro)
# O padrão r'(\d+)' captura '1703' se não houver um caractere oculto.
tabela_quilombola['Populacao_Numerica'] = tabela_quilombola['População'].str.extract(r'(\d+)', expand=False)

# 3. Conversão Final para Tipo Numérico
# Converte a coluna para o tipo numérico, substituindo falhas (como "Sem informação" ou extrações incompletas) por NaN.
tabela_quilombola['Populacao_Numerica'] = pd.to_numeric(
    tabela_quilombola['Populacao_Numerica'], 
    errors='coerce'
)

# 4. Limpeza da População Incompleta:
# Comunidades onde a extração falhou (extraindo '170' em vez de '1703') ou extraiu 'Sem Informação'
# serão representadas como NaN, conforme desejado para a análise.

# 5. Visualização do Resultado
print("--- Amostra Corrigida do DataFrame com a nova coluna numérica ---")
# Verifique se o valor 1703 e outros números aparecem corretamente.
print(tabela_quilombola[['Comunidade(s)', 'População', 'Populacao_Numerica']].head(10).to_markdown(index=False))

--- Amostra Corrigida do DataFrame com a nova coluna numérica ---
| Comunidade(s)                                                                                        | População                                            |   Populacao_Numerica |
|:-----------------------------------------------------------------------------------------------------|:-----------------------------------------------------|---------------------:|
| Itabatinga, Mangabeira, Porto Grande, Santo Antônio de Vizeu, São Benedito de Viseu, Uxizal, Vizânia | 1703 pessoas (Censo, 2022); 400 famílias             |                 1703 |
| Abacatal – Aurá                                                                                      | 368 pessoas (Censo, 2022) ; 53 famílias              |                  368 |
| Abelha                                                                                               | Sem informação                                       |                  nan |
| Abobral Margem Di

In [4]:
# Contagem total de comunidades
total_comunidades = len(tabela_quilombola)

# Lista das colunas que queremos analisar (incluindo a nova e limpa)
colunas_analise = [
    'Populacao_Numerica', 
    'Município', 
    'Unidade da federação', 
    'Situação fundiária', 
    'Data da última atualização'
]

print("\n--- Resultados da Análise Estatística de Completude ---")
print(f"Total de comunidades (registros): {total_comunidades}")
print("-" * 60)
print("ANÁLISE DE DADOS FALTANTES POR CAMPO:")
print("-" * 60)

# Análise de cada coluna
for coluna in colunas_analise:
    qnt_faltante = tabela_quilombola[coluna].isnull().sum()
    pct_faltante = (qnt_faltante / total_comunidades) * 100
    
    # Ajusta o nome da coluna para exibição no relatório
    nome_exibicao = "População (Numérica)" if coluna == 'Populacao_Numerica' else coluna
    
    print(f"{nome_exibicao.ljust(25)} | Qtd. Faltante: {str(qnt_faltante).ljust(5)} | % Faltante: {pct_faltante:.2f}%")

# --- Análise Geral (Linhas Completamente Vazias) ---

# Calcula quantas linhas têm PELO MENOS um valor faltante em qualquer coluna do nosso subconjunto
qnt_com_dados_ausentes_geral = tabela_quilombola[colunas_analise].isnull().any(axis=1).sum()
pct_com_dados_ausentes_geral = (qnt_com_dados_ausentes_geral / total_comunidades) * 100

print("-" * 60)
print("ANÁLISE GERAL DE COMPLETUDE:")
print("-" * 60)
print(f"Registros Incompletos (pelo menos 1 campo faltante): {qnt_com_dados_ausentes_geral} registros")
print(f"Porcentagem de Registros Incompletos: {pct_com_dados_ausentes_geral:.2f}%")
print("-" * 60)


--- Resultados da Análise Estatística de Completude ---
Total de comunidades (registros): 2115
------------------------------------------------------------
ANÁLISE DE DADOS FALTANTES POR CAMPO:
------------------------------------------------------------
População (Numérica)      | Qtd. Faltante: 1452  | % Faltante: 68.65%
Município                 | Qtd. Faltante: 145   | % Faltante: 6.86%
Unidade da federação      | Qtd. Faltante: 146   | % Faltante: 6.90%
Situação fundiária        | Qtd. Faltante: 145   | % Faltante: 6.86%
Data da última atualização | Qtd. Faltante: 145   | % Faltante: 6.86%
------------------------------------------------------------
ANÁLISE GERAL DE COMPLETUDE:
------------------------------------------------------------
Registros Incompletos (pelo menos 1 campo faltante): 1452 registros
Porcentagem de Registros Incompletos: 68.65%
------------------------------------------------------------


### Produção do relatório.

In [5]:
import pandas as pd
import numpy as np
import io
import os

# ⚠️ É crucial que 'tabela_quilombola' tenha a coluna 'Populacao_Numerica'
# e que esta coluna já tenha seus valores vazios ou não numéricos como NaN.

# --- 1. Limpeza e Normalização dos Dados (Inclui Populacao_Numerica) ---

# Cria uma cópia para análise
df_analise = tabela_quilombola.copy()

# Garante que qualquer string vazia em TODAS as colunas (incluindo as originais) vire NaN.
df_analise = df_analise.replace(r'^\s*$', np.nan, regex=True)

# Lista das colunas principais para análise de completude
# Substituímos 'População' pela coluna numérica, mais relevante para a análise.
colunas_analise = [
    'Populacao_Numerica', 
    'Município', 
    'Unidade da federação', 
    'Situação fundiária', 
    'Data da última atualização'
]

# --- 2. Cálculos Estatísticos ---

total_comunidades = len(df_analise)

# Usa um objeto StringIO para construir o relatório em memória
relatorio = io.StringIO()
relatorio.write("=" * 60 + "\n")
relatorio.write("RELATÓRIO DE COMPLETUDE DOS DADOS QUILOMBOLAS (CPISP)\n")
relatorio.write("=" * 60 + "\n")
relatorio.write(f"Total de Registros Coletados: {total_comunidades}\n\n")

# --- 3. Análise Detalhada por Coluna ---

relatorio.write("-" * 60 + "\n")
relatorio.write("Análise da Ausência de Dados por Coluna:\n")
relatorio.write("-" * 60 + "\n")

for coluna in colunas_analise:
    qnt_faltante = df_analise[coluna].isnull().sum()
    pct_faltante = (qnt_faltante / total_comunidades) * 100
    
    # Ajusta o nome da coluna para exibição no relatório
    nome_exibicao = "População (Numérica)" if coluna == 'Populacao_Numerica' else coluna
    
    relatorio.write(f"Coluna: {nome_exibicao}\n")
    relatorio.write(f"  > Registros SEM informação: {qnt_faltante}\n")
    relatorio.write(f"  > Porcentagem SEM informação: {pct_faltante:.2f}%\n")
    relatorio.write("-" * 20 + "\n")

# --- 4. Análise da Linha Completa ---

# Calcula quantas linhas que possuem ao menos um dado faltante NO SUBSET DE ANÁLISE
qnt_linhas_incompletas = df_analise[colunas_analise].isnull().any(axis=1).sum()
pct_linhas_incompletas = (qnt_linhas_incompletas / total_comunidades) * 100

relatorio.write("\n" + "=" * 60 + "\n")
relatorio.write("Análise da Completude do Registro (Geral):\n")
relatorio.write("=" * 60 + "\n")
relatorio.write(f"Total de Comunidades com Algum Dado Faltante (no subset): {qnt_linhas_incompletas}\n")
relatorio.write(f"Porcentagem de Registros Incompletos (no subset): {pct_linhas_incompletas:.2f}%\n")
relatorio.write("-" * 60 + "\n")

# Pega o conteúdo completo do relatório
relatorio_final = relatorio.getvalue()

# --- 5. Salvamento do Relatório ---

nome_arquivo_relatorio = 'relatorio_completude_quilombolas.txt'

try:
    with open(nome_arquivo_relatorio, 'w', encoding='utf-8') as f:
        f.write(relatorio_final)
    
    print(f"✅ Relatório estatístico salvo com sucesso como: {nome_arquivo_relatorio}")

except Exception as e:
    print(f"❌ Erro ao salvar o relatório: {e}")
    print("Ocorreu um erro de salvamento. Exibindo o relatório no console.")
    print("\n--- Conteúdo do Relatório ---")
    print(relatorio_final)

✅ Relatório estatístico salvo com sucesso como: relatorio_completude_quilombolas.txt


In [6]:
soma_populacao = tabela_quilombola['Populacao_Numerica'].sum()
print(soma_populacao)

132030.0


In [7]:
tabela_quilombola.to_csv('dados_quilombolas_cpisp_final.csv', index=False, encoding='utf-8')